<a href="https://colab.research.google.com/github/Aneesha0306/plant_detection/blob/main/plant_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

zip_file_path='/content/drive/MyDrive/college/yellow mosaic.zip'

In [ ]:
unzip_dir = '/content/drive/MyDrive/plant_leafpic'

In [ ]:
if not os.path.exists(unzip_dir):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(unzip_dir)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
img_size = 150  # Resize images to 150x150
batch_size = 32

In [ ]:
base_dir = unzip_dir

In [ ]:
# Create ImageDataGenerator for training and validation sets
data_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Use 20% of data for validation
)

In [ ]:
# Train Generator (80% of data for training)
train_generator = data_gen.flow_from_directory(
    base_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='training',
    class_mode='binary'  # 2 classes: healthy vs yellow mosaic
)

Found 1056 images belonging to 2 classes.


In [ ]:
# Validation Generator (20% of data for validation)
validation_generator = data_gen.flow_from_directory(
    base_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='validation',
    class_mode='binary'  # 2 classes: healthy vs yellow mosaic
)

Found 263 images belonging to 2 classes.


In [ ]:
# Define the Model
from tensorflow.keras import layers, models

model = models.Sequential()

In [ ]:
# First Convolutional Layer
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))
model.add(layers.MaxPooling2D(2, 2))

In [ ]:
# Second Convolutional Layer
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))

In [ ]:
# Third Convolutional Layer
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(2, 2))

In [ ]:
# Flatten the output from convolutional layers
model.add(layers.Flatten())

In [ ]:
# Fully connected layer
model.add(layers.Dense(256, activation='relu'))

In [ ]:
# Output layer (1 neuron for binary classification)
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)                    │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       9,470,208 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,563,713 (36.48 MB)

 Trainable params: 9,563,713 (36.48 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=5,  # You can increase epochs if needed
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 14s 229ms/step - accuracy: 0.5968 - loss: 1.2205 - val_accuracy: 0.6875 - val_loss: 0.5471
Epoch 2/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 7s 210ms/step - accuracy: 0.7051 - loss: 0.5429 - val_accuracy: 0.9062 - val_loss: 0.2557
Epoch 3/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 10s 210ms/step - accuracy: 0.9115 - loss: 0.2514 - val_accuracy: 0.9258 - val_loss: 0.2171
Epoch 4/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 6s 195ms/step - accuracy: 0.9362 - loss: 0.1783 - val_accuracy: 0.9102 - val_loss: 0.2940
Epoch 5/5
33/33 ━━━━━━━━━━━━━━━━━━━━ 11s 216ms/step - accuracy: 0.9502 - loss: 0.1522 - val_accuracy: 0.9375 - val_loss: 0.1442


In [ ]:
model.save('/content/drive/MyDrive/plant_disease_prediction_model.h5')

In [ ]:
import json
class_indices = train_generator.class_indices  # Get class indices from training generator
json.dump(class_indices, open('/content/drive/MyDrive/class_indices.json', 'w'))

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
def predict_image_class(model, img_path, class_indices):
    img = image.load_img(img_path, target_size=(img_size, img_size))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict class (0: yellow mosaic, 1: healthy)
    prediction = model.predict(img_array)
    predicted_class = 1 if prediction[0] > 0.5 else 0

    class_names = {v: k for k, v in class_indices.items()}
    predicted_class_name = class_names[predicted_class]

    return predicted_class_name

In [ ]:
# test with sample image
image_path = '/content/drive/MyDrive/college/test_leaf.jpg'
predicted_class_name = predict_image_class(model, image_path, class_indices)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 616ms/step


In [ ]:
print("Predicted Class Name:", predicted_class_name)

Predicted Class Name: yellow mosaic
